In [1]:
import pandas as pd

In [88]:
costs = (
    pd
    .read_excel('../data/system_inputs.xlsx', sheet_name='technology_params')
    .drop(['CO2 intensity'],axis=1)
    .set_index('Row Labels')
    .fillna(0)
)

costs['capex'] = costs['investment']

costs = costs.drop(['investment'],axis=1)

costs.head(5)

,efficiency,FOM,fuel,lifetime,VOM,capex
Row Labels,,,,,,
battery inverter,0.900,3.000000,0.0,20,0.0,411.0
battery storage,0.000,0.000000,0.0,15,0.0,192.0
biomass,0.468,4.526935,7.0,30,0.0,2209.0
gas,0.500,2.500000,21.6,30,4.0,800.0
coal,0.464,1.923076,8.4,40,6.0,1300.0


## Calculating capital costs
The capital costs in the dataframe above are the **overnight** capital costs provided in USD/MW. Overnight cost is the cost of a construction project if no interest was incurred during construction, as if the project was completed "overnight". We therefore adjust the overnight capital costs by annualising them to net present costs with a discount rate $r$ over the economic lifetime of the asset $n$ using the annuity factor $a$, such that:

$$
a = \frac{1-(1+r)^{-n}}{r}
$$

### Define annuity function:

In [89]:
def calculate_annuity(
        n : int, 
        r : float,
) -> float:
    '''

    Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20, 0.05) * 20 = 1.6
    
    Inputs:
    -----------------------------------

        n : asset lifetime (years)
        r : discount rate (%, decimal point [e.g., 0.04])

    '''

    if isinstance(r, pd.Series):
        return pd.Series(1/n, index=r.index).where(r == 0, r/(1. - 1./(1.+r)**n))
    elif r > 0:
        return r/(1. - 1./(1.+r)**n)
    else:
        return 1/n

In [93]:
# Compute: Capital costs
optimisation_years = 2030-2024
discount_rate = 0.05

costs["annualised_capex"] = (
    (
        calculate_annuity(costs["lifetime"], discount_rate) + costs["FOM"] / 100.0
    )
    * costs["capex"]
    * optimisation_years
).fillna(0)

costs['annualised_capex_uom'] = 'USD/MW/annum'

costs.loc[['coal','solar','wind']]

,efficiency,FOM,fuel,lifetime,VOM,capex,annualised_capex,annualised_capex_uom
Row Labels,,,,,,,,
coal,0.464,1.923076,8.4,40.0,6.0,1300.0,604.569585,USD/MW/annum
solar,0.000,3.000000,0.0,25.0,0.0,425.0,257.428766,USD/MW/annum
wind,0.000,2.450549,0.0,30.0,2.3,1040.0,558.835213,USD/MW/annum


## Calculating marginal costs
The marginal cost $c_m$ is a function of the variable operational and maintenance costs ($c_{vom}$), fuel costs ($c_f$) and the efficiency $e$, such that:

$$
c_m = c_{vom} + \frac{c_{f}}{e}
$$

In [102]:
# now let's compute the marginal cost
costs["marginal_cost"] = (costs["VOM"] + costs["fuel"] / costs["efficiency"]).fillna(0)

costs["marginal_cost_uom"] = 'USD/MWh'

costs.head(5)

,efficiency,FOM,fuel,lifetime,VOM,capex,annualised_capex,annualised_capex_uom,marginal_cost,marginal_cost_uom
Row Labels,,,,,,,,,,
battery inverter,0.900,3.000000,0.0,20.0,0.0,411.0,271.858220,USD/MW/annum,0.000000,USD/MWh
battery storage,0.000,0.000000,0.0,15.0,0.0,192.0,110.986315,USD/MW/annum,0.000000,USD/MWh
biomass,0.468,4.526935,7.0,30.0,0.0,2209.0,1462.191685,USD/MW/annum,14.957265,USD/MWh
gas,0.500,2.500000,21.6,30.0,4.0,800.0,432.246888,USD/MW/annum,47.200000,USD/MWh
coal,0.464,1.923076,8.4,40.0,6.0,1300.0,604.569585,USD/MW/annum,24.103448,USD/MWh


In [103]:
# calculating captial costs of storage is a little trickier... let's do that now
def costs_for_storage(
        # calculate cost of storages
        store, link1, link2=None, max_hours=1.0
):
    capital_cost = link1["annualised_capex"] + max_hours * store["annualised_capex"]
    if link2 is not None:
        capital_cost += link2["annualised_capex"]
    return pd.Series(
        dict(annualised_capex=capital_cost, marginal_cost=0.0)
    )

# compute storage costs
costs.loc["battery"] = costs_for_storage(
    costs.loc["battery storage"],
    costs.loc["battery inverter"],
    max_hours=4, # assuming 4-hr battery storage
)

# compute H2 storage costs
costs.loc["H2"] = costs_for_storage(
    costs.loc["hydrogen underground storage"],
    costs.loc["fuel cell"],
    costs.loc["electrolysis"],
    max_hours=168, # assuming 168-hr H2 storage
)

costs.loc[['battery','H2']]

,efficiency,FOM,fuel,lifetime,VOM,capex,annualised_capex,annualised_capex_uom,marginal_cost,marginal_cost_uom
Row Labels,,,,,,,,,,
battery,NaN,NaN,NaN,NaN,NaN,NaN,715.803481,NaN,0.0,NaN
H2,NaN,NaN,NaN,NaN,NaN,NaN,517.252682,NaN,0.0,NaN


### Now, let's assume we are costing up a 1000 MW solar farm and apply a social discount rate

In [106]:
plant_capacity = 1000 # MW
n_years = 2030-2024 # years

# get solar capex (USD/MW/annum)
solar_capex = costs.loc['solar'].annualised_capex

# compute annual cost (USD/annum)
total_annual_cost =  solar_capex * plant_capacity 

# convert to millions
total_annual_cost *= 1e-6

# convert to present value applying a social discount rate
social_discount_rate = 0.1

total_annual_cost / (1 + social_discount_rate)**(n_years)

0.14531182731671974